# HOT Tasking Manager project analysis
In this notebook we demonstrate how to combine two datasets: HOT Tasking Manager project data and OSM contributions.

We want to investigate all contributions that have been made in OSM via HOT's Tasking Manager. We will furthermore filter these contributions by using the **OSM Changeset** information.

These are the steps you see further down:
* Set the connection parameters.
* Prepare your input parameters, e.g. define HOT Tasking Manager project ID.
* Download data using DuckDB. This time we also download HOT project data.
* Filter OSM contributions using changeset attributes.
* Display both datasets on a map.

## Getting started
Set connection params.

In [1]:
import os

s3_user = os.environ["S3_ACCESS_KEY_ID"]  # add your user here
s3_password = os.environ["S3_SECRET_ACCESS_KEY"]  # add your password here

Configure DuckDB.

In [2]:
import duckdb

con = duckdb.connect(
    config={
        'threads': 8,
        'max_memory': '8GB',
        # 'enable_object_cache': True
    }
)
con.install_extension("spatial")
con.load_extension("spatial")

Set the connection params to Iceberg Rest Catalog. (We need this for the OSM data.)

In [4]:
from pyiceberg.catalog.rest import RestCatalog

catalog = RestCatalog(
    name="default",
    **{
        "uri": "https://sotm2024.iceberg.ohsome.org",
        "s3.endpoint": "https://sotm2024.minio.heigit.org",
        "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
        "s3.access-key-id": s3_user,
        "s3.secret-access-key": s3_password,
        "s3.region": "eu-central-1"
    }
)

Set connection to MinIO object storage. (We need this for HOT Tasking Manager data)

In [5]:
query = f"""
DROP SECRET IF EXISTS "__default_s3";
CREATE SECRET (
      TYPE S3,
      KEY_ID '{s3_user}',
      SECRET '{s3_password}',
      REGION 'eu-central-1',
      endpoint 'sotm2024.minio.heigit.org',
      use_ssl true,
      url_style 'path'
  );
"""
con.sql(query).show()

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘



## Prepare the input parameters for your analysis

In [6]:
hot_tm_parquet_data_path = "s3a://heigit-ohsome-sotm24/data/hot_tasking_manager/**"

# Set iceberg table
namespace = 'geo_sort'
tablename = 'contributions'
icebergtable = catalog.load_table((namespace, tablename))

hot_tm_project_id = 15996

## Get Information about HOT Tasking Manager project

In [7]:
import geopandas as gpd

query = f"""
SELECT *
FROM read_parquet('{hot_tm_parquet_data_path}') a
WHERE project_id = {hot_tm_project_id};
"""
df = con.sql(query).df()

bbox = df["bbox"].values[0]

# convert the data to geodata
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.GeoSeries.from_wkt(df['geometry'])
).set_crs('epsg:4326')

Display where the project is located.

In [8]:
import lonboard

# the lonboard map definition
hot_tm_layer = lonboard.PolygonLayer.from_geopandas(
    gdf,
    get_fill_color=[0, 0, 0, 0],
    get_line_color=[255, 0, 255],
    get_line_width=10
)

view_state = {
    "longitude": df["centroid"].values[0]["x"],
    "latitude": df["centroid"].values[0]["y"],
    "zoom": 12
}

overview_map = lonboard.Map(
    basemap_style=lonboard.basemap.CartoBasemap.Positron,
    layers=[hot_tm_layer],
    view_state=view_state
)

display(overview_map)

Map(basemap_style=<CartoBasemap.Positron: 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json'>, la…

## Download data OSM Data

In [10]:
import time
start_time = time.time()


icebergtable.scan(
    row_filter=(
        f"( status = 'latest' or status = 'history' )"
        f"and (xmax >= {bbox['xmin']} and xmin <= {bbox['xmax']}) "
        f"and (ymax >= {bbox['ymin']} and ymin <= {bbox['ymax']}) "
    ),
    selected_fields=(
        "user_id",
        "osm_id",
        "osm_version",
        "valid_from",
        "valid_to",
        "tags",
        "tags_before",
        "changeset",
        "geometry",
    ),
).to_duckdb('raw_osm_data',connection=con)

download_time = round(time.time() - start_time, 3)
print(f"download took {download_time} sec.")

download took 58.948 sec.


## Filter data using OSM changeset information

In [11]:
query = f"""
    SELECT *
    FROM raw_osm_data
    WHERE 1=1
        and list_contains(changeset.hashtags, 'hotosm-project-{hot_tm_project_id}')
    """
osm_df = con.sql(query).df()

# convert the data to geodata
osm_gdf = gpd.GeoDataFrame(
    osm_df,
    geometry=gpd.GeoSeries.from_wkt(osm_df['geometry'])
).set_crs('epsg:4326')

Display OSM data and HOT Tasking Manager project on map.

In [12]:
# the lonboard map definition
osm_polygons_layer = lonboard.PolygonLayer.from_geopandas(
    osm_gdf[osm_gdf.geom_type == 'Polygon'],
    get_fill_color=[0, 0, 0, 0],
    get_line_width=1
)

osm_lines_layer = lonboard.PathLayer.from_geopandas(
    osm_gdf[osm_gdf.geom_type == 'LineString'],
)


osm_points_layer = lonboard.ScatterplotLayer.from_geopandas(
    osm_gdf[osm_gdf.geom_type == 'Point'],
)



osm_map = lonboard.Map(
    basemap_style=lonboard.basemap.CartoBasemap.Positron,
    layers=[
        hot_tm_layer,
        osm_polygons_layer,
        osm_lines_layer,
        osm_points_layer,
    ],
    view_state=view_state
)

display(osm_map)

Map(basemap_style=<CartoBasemap.Positron: 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json'>, la…

## Display OSM user statistics
Let's analyse who mapped how much in this area.

In [13]:
query = f"""
    SELECT
        user_id,
        count(*) as n_edits,
        SUM(CASE
            WHEN contains(changeset.editor, 'JOSM') THEN 1
            ELSE 0
        END) as josm_edit,
        SUM(CASE
            WHEN contains(changeset.editor, 'iD') THEN 1
            ELSE 0
        END) as iD_edit
    FROM raw_osm_data
    WHERE 1=1
        and list_contains(changeset.hashtags, 'hotosm-project-{hot_tm_project_id}')
    GROUP BY user_id
    ORDER BY n_edits DESC
    """
user_stats_df = con.sql(query).df()

We are going to add OSM usernames to our dataframe.

In [14]:
import requests


def get_current_usernames(user_ids_list):
    api_url = "http://textual.ru/whosthat/whosthat.php"
    user_names_list = []

    for user_id in user_ids_list:
        params = {
            "action": "info",  # Use "info" action to get detailed information
            "id": user_id,
        }
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if data and isinstance(data, list) and len(data) > 0:
                # The API returns an array of hashes with user information
                # We need to extract the user ID from the first entry
                user_info = data[0]
                user_names_list.append( user_info['names'][-1]["name"] )

    return user_names_list


user_stats_df["username"] = get_current_usernames(user_stats_df["user_id"].values)
user_stats_df["link"] = "https://www.openstreetmap.org/user/" + user_stats_df["username"]

Display a user leaderboard.

In [16]:
display(user_stats_df)

,user_id,n_edits,josm_edit,iD_edit,username,link
0,563947,898,0.0,858.0,notoncebut2x,https://www.openstreetmap.org/user/notoncebut2x
1,11010237,734,734.0,0.0,Dannyoung,https://www.openstreetmap.org/user/Dannyoung
2,10570996,468,468.0,0.0,Erick tamba,https://www.openstreetmap.org/user/Erick tamba
3,21292317,439,0.0,439.0,abhimanyubarun,https://www.openstreetmap.org/user/abhimanyubarun
4,17340053,388,388.0,0.0,Sally Mlingi,https://www.openstreetmap.org/user/Sally Mlingi
5,13722972,261,186.0,75.0,Jimerveille,https://www.openstreetmap.org/user/Jimerveille
6,21288883,251,0.0,251.0,BTR2,https://www.openstreetmap.org/user/BTR2
7,19063413,245,0.0,245.0,Aczel-Divin MOUNTOU,https://www.openstreetmap.org/user/Aczel-Divin...
8,20450590,167,0.0,167.0,KOGNOKO ABOULATSAMBO,https://www.openstreetmap.org/user/KOGNOKO ABO...
9,9449463,150,150.0,0.0,Mercy14846,https://www.openstreetmap.org/user/Mercy14846


Inspect the contributions made by the mapper with most edits.

In [18]:
top_mapper = user_stats_df.sort_values("n_edits")["user_id"][0]

# the lonboard map definition
osm_polygons_layer = lonboard.PolygonLayer.from_geopandas(
    osm_gdf[(osm_gdf.geom_type == 'Polygon') & (osm_gdf["user_id"] == top_mapper)],
    get_fill_color=[0, 0, 0, 0],
    get_line_width=1
)

osm_lines_layer = lonboard.PathLayer.from_geopandas(
    osm_gdf[(osm_gdf.geom_type == 'LineString') & (osm_gdf["user_id"] == top_mapper)],
)


osm_points_layer = lonboard.ScatterplotLayer.from_geopandas(
    osm_gdf[(osm_gdf.geom_type == 'Point') & (osm_gdf["user_id"] == top_mapper)],
)



osm_map = lonboard.Map(
    basemap_style=lonboard.basemap.CartoBasemap.Positron,
    layers=[
        hot_tm_layer,
        osm_polygons_layer,
        osm_lines_layer,
        osm_points_layer,
    ],
    view_state=view_state
)

display(osm_map)

Map(basemap_style=<CartoBasemap.Positron: 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json'>, la…